# Rolling Average Trading Strategy with Python

The purpose of this script is to implement a very simple trading strategy with Python. This strategy is based on the difference between slow and fast moving averages on the adjusted closed prices of shares. First we describe the strategy in detail explaining various features and then we optimize it using Python libraries.  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from itertools import product

warnings.filterwarnings("ignore")

As an example BTCUSDT data was chosen

In [ ]:
stock = "BTCUSDT"
data = pd.read_csv(f"../data/{stock}.csv")
data["log_returns"] = np.log(data["close"] / data["close"].shift(1))
data.dropna(inplace=True)
data.head(5)
# data.to_csv('data.cvs') If we want to save the data

The close price during that period looks like this:

In [ ]:
plt.figure(figsize=(15, 9))
data["close"].plot(c="g")
plt.ylabel("Close Price")
plt.show()

And here we plot the cumulative log returns $c_{t}$ and the total relative returns $c_{t}^{rel}$, given by:
$$
c_{t} = \sum_{k = 0}^{t} \ln(p_{k}/p_{k-1})\,,
$$
and
$$
c_{t}^{rel} = \exp(c_{t}) - 1
$$

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10))

ax1.plot(data["log_returns"].cumsum(), c="r")
ax1.set_ylabel("Cumulative Log Returns")

ax2.plot(100 * (np.exp(data["log_returns"].cumsum()) - 1))
ax2.set_ylabel("Total Relative Returns %")

plt.show()

In [ ]:
# Compute a fast moving average of 40 days (MA40) and a slow moving
# average of 252 days (MA252) on the close price.
MAF = 40
MAS = 252
data["ma_slow"] = data["close"].rolling(MAS).mean()
data["ma_fast"] = data["close"].rolling(MAF).mean()

# Plot the close price, the fast moving average and the slow moving average
plt.figure(figsize=(15, 9))
data["close"].plot(color="g", label="Close Price")
data["ma_fast"].plot(color="r", label="MA{}".format(MAF))
data["ma_slow"].plot(color="b", label="MA{}".format(MAS))
plt.legend()
plt.show()

We will now follow a simple strategy: if M45 > M252, we then long one share of stock; otherwise, we do nothing. Our signal is then defined as 

In [ ]:
data["signal"] = np.where(data["ma_fast"] > data["ma_slow"], 1, -1)
data.head()

We see that, I we were to follow this strategy, we would go long more often than short:

In [ ]:
sns.countplot(x=data["signal"])
print(data["signal"].value_counts())

And here plot the close price for our stock, the slow/fast moving averages and the short/long signal:

In [ ]:
Ax = data[["close", "ma_fast", "ma_slow", "signal"]].plot(
    secondary_y="signal",
    figsize=(15, 9),
    title="Close Price for {}, Slow/Fast Moving Averages ({} and {}) and Short/Long signal".format(
        stock, MAS, MAF
    ),
)
Ax.get_legend().set_bbox_to_anchor((1.17, 0.85))

Following this strategy, the daily returns $R_{t}^{S}$ would be given by the product of the log return $R_{t}$ times our signal $O_{t}$, like so:
$$
R^{S}_{t} = O_{t} R_{t}
$$

In [ ]:
data["strg_log_returns"] = data["log_returns"] * data["signal"]
data.dropna(inplace=True)
data.head()

(We note that we have shifted the signal for one day, since we implement the strategy on the day after the closing.)

The results of the strategy are not encouraging. We compare it to a 'Buy and Hold' strategy:

In [ ]:
plt.figure(figsize=(15, 9))
data["strg_log_returns"].cumsum().plot(label="Cumulative log_returns (Strategy)")
data["log_returns"].cumsum().plot(label="Cumulative log_returns (Buy and Hold)")
plt.title("Cumulative log_returns")
plt.legend()
plt.show()

So, for this share the strategy underperforms and it is better to stick to Buy and Hold.

To evaluate the strategy better, we compute the total returns for the strategy and the market cases, as well as the annualized volatility:

In [ ]:
print("Total Returns:")
print(
    "Market Total Return:",
    ((np.exp(data[["log_returns", "strg_log_returns"]].sum()) - 1) * 100)
    .round(3)
    .iloc[0],
    "%",
)
print(
    "Strategy Total Return:",
    ((np.exp(data[["log_returns", "strg_log_returns"]].sum()) - 1) * 100)
    .round(3)
    .iloc[1],
    "%",
)
print(15 * "===")
print("Annualized Volatility:")
print(
    "Market Volatility:",
    (data[["log_returns", "strg_log_returns"]].std() * 365**0.5).round(5).iloc[0],
    "%",
)
print(
    "Strategy Volatility:",
    (data[["log_returns", "strg_log_returns"]].std() * 365**0.5).round(5).iloc[1],
    "%",
)

In [ ]:
def MA_Strategy(stock, MAF, MAS, showPlot=True):
    data = pd.read_csv("../data/{}.csv".format(stock))
    data["log_returns"] = np.log(data["close"] / data["close"].shift(1))
    data.dropna(inplace=True)
    data["ma_slow"] = data["close"].rolling(MAS).mean()
    data["ma_fast"] = data["close"].rolling(MAF).mean()
    data.dropna(inplace=True)
    data["signal"] = np.where(data["ma_fast"] > data["ma_slow"], 1, -1)
    data.dropna(inplace=True)
    data["strg_log_returns"] = data["log_returns"] * data["signal"].shift(1)
    data.dropna(inplace=True)

    # We show the results:

    data[["close", "ma_fast", "ma_slow", "signal"]].plot(
        secondary_y="signal",
        figsize=(15, 9),
        title="Close Price for {}, Slow/Fast Moving Averages ({} and {}) and Short/Long signal".format(
            stock[0], MAS, MAF
        ),
    ).get_legend().set_bbox_to_anchor((1.17, 0.85))

    if showPlot:
        plt.figure(figsize=(15, 9))
        (100 * (np.exp(data["strg_log_returns"].cumsum()) - 1)).plot(
            label="Cumulative Returns (Strategy)"
        )
        (100 * (np.exp(data["log_returns"].cumsum()) - 1)).plot(
            label="Cumulative Returns (Buy and Hold)"
        )
        plt.title("Cumulative Returns (%): {}".format(stock))
        plt.legend()
        plt.show()

    print("\n")
    print("Total Returns:", stock)
    print(
        "Market Total Return:",
        ((np.exp(data[["log_returns", "strg_log_returns"]].sum()) - 1) * 100)
        .round(3)
        .iloc[0],
        "%",
    )
    print(
        "Strategy Total Return:",
        ((np.exp(data[["log_returns", "strg_log_returns"]].sum()) - 1) * 100)
        .round(3)
        .iloc[1],
        "%",
    )
    print(15 * "===")
    print("Annualized Volatility:", stock)
    print(
        "Market Volatility:",
        (data[["log_returns", "strg_log_returns"]].std() * 365**0.5).round(5).iloc[0],
        "%",
    )
    print(
        "Strategy Volatility:",
        (data[["log_returns", "strg_log_returns"]].std() * 365**0.5).round(5).iloc[1],
        "%",
    )

In [ ]:
stock = "BTCUSDT"

MA_Strategy(stock, MAF, MAS)

In [ ]:
stock = "ETHUSDT"
MA_Strategy(stock, MAF, MAS)

In [ ]:
# Optimization by choosing the best parameters
sma1 = range(10, 90, 5)
sma2 = range(100, 250, 10)
stock = "BTCUSDT"

show = pd.DataFrame()

for ma_fast, ma_slow in product(sma1, sma2):
    data = pd.read_csv("../data/{}.csv".format(stock))
    data.dropna(inplace=True)
    data["log_returns"] = np.log(data["close"] / data["close"].shift(1))
    data.dropna(inplace=True)
    data["ma_slow"] = data["close"].rolling(ma_slow).mean()
    data["ma_fast"] = data["close"].rolling(ma_fast).mean()
    data.dropna(inplace=True)
    data["signal"] = np.where(data["ma_fast"] > data["ma_slow"], 1, -1)
    data["strg_log_returns"] = data["log_returns"] * (data["signal"].shift(1))
    data.dropna(inplace=True)

    ret = (np.exp(data[["log_returns", "strg_log_returns"]].sum()) - 1) * 100
    info = {
        "MA Slow": ma_slow,
        "MA Fast": ma_fast,
        "Market Return (%)": (
            (np.exp(data[["log_returns", "strg_log_returns"]].sum()) - 1) * 100
        )
        .iloc[0]
        .round(3),
        "Strategy Return (%)": (
            (np.exp(data[["log_returns", "strg_log_returns"]].sum()) - 1) * 100
        )
        .iloc[1]
        .round(3),
        "Difference in Returns (%)": (
            ret["strg_log_returns"] - ret["log_returns"]
        ).round(3),
    }

    show = pd.concat([show, pd.DataFrame(info, index=[0])], ignore_index=True)

In [ ]:
top5 = show.sort_values("Difference in Returns (%)", ascending=False).head(5)
display(top5)

In [ ]:
# Take a look on the best strategy
MAS = top5["MA Slow"].iloc[0]
MAF = top5["MA Fast"].iloc[0]

MA_Strategy(stock, MAF, MAS)